<a href="https://colab.research.google.com/github/gatherheart/Classification_With_BERT/blob/main/news_classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 55.0MB 103kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 348kB 8.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588512 sha256=59b65a851accd2442cb00486080a3d66df46c5dc98dc760ef18809203ff036da
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
     |████████████████████████████████| 1.1MB 9.1MB/s 
     |████████████████████████████████| 757kB 8.0MB/s 
     |████████████████████████████████| 3.0MB 16.9MB/s 
     |████████████████████████████████| 890kB 61.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=a5a3db2530edb489fd5ed765bbb60777179d198cea6b34262b24642d135e9b9a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a501

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-rdr2tnsv
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-rdr2tnsv
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=47162fd1582ab7164b5846177428cb288c8fe3b60575b8be0de53fd58d552044
  Stored in directory: /tmp/pip-ephem-wheel-cache-px6deap3/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook, notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
import tensorflow as tf

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [8]:
##GPU 사용 시
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [10]:
BASE_PATH = '/content/drive/MyDrive/hyun_data'

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Dec 12 04:40:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
df = pd.read_csv(BASE_PATH+'/preprocessed_norm.csv', index_col=[0])

In [12]:
labels_csv = pd.read_csv(BASE_PATH+'/labels_dict2.csv', index_col=[0])

In [13]:
labels_csv

,문화>방송_연예,국제>중국,사회>사회일반,정치>국회_정당,사회>교육_시험,IT_과학>모바일,정치>행정_자치,사회>사건_사고,문화>문화일반,정치>청와대,사회>여성,국제>유럽_EU,경제>자원,국제>미국_북미,사회>날씨,사회>의료_건강,IT_과학>콘텐츠,경제>증권_증시,정치>선거,국제>일본,사회>미디어,문화>음악,국제>중남미,국제>중동_아프리카,경제>유통,문화>출판,경제>자동차,사회>환경,경제>금융_재테크,사회>장애인,문화>전시_공연,경제>부동산,IT_과학>IT_과학일반,문화>요리_여행,IT_과학>과학,경제>서비스_쇼핑,경제>외환,미분류,문화>미술_건축,문화>생활,경제>경제일반,국제>아시아,정치>정치일반,경제>국제경제,경제>취업_창업,IT_과학>보안,정치>외교,경제>무역,문화>영화,정치>북한,경제>산업_기업,문화>종교,경제>반도체,문화>학술_문화재,사회>노동_복지,국제>국제일반,IT_과학>인터넷_SNS,국제>러시아
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57


In [14]:
df.columns

Index(['date', 'title', 'topic', 'keyword', 'input', 'body', 'label'], dtype='object')

In [15]:
df[df.label==0]

,date,title,topic,keyword,input,body,label
8,20201128,"'온앤오프' 이지아, '데뷔 후 최초' 베일 싸여있던 일상 공개",문화>방송_연예,"온앤오프,이지아,데뷔,베일,일상,공개,28일,방송,tvN,온앤오프,배우,이지아,반전...",이지아 온앤오프 덕후 한경닷컴 김나경 tvn 은하계 ufo 동작 하나하나 외계인설 ...,28일(토) 방송하는 tvN ’온앤오프‘에서는 배우 이지아의 반전 매력 넘치는 일상...,0
10,20201128,"'런닝맨' 김소연, '팬심' 폭발시킨 멤버 정체는?",문화>방송_연예,"런닝맨,김소연,팬심,폭발,멤버,정체,배우,김소연,팬심,고백,29일,방송,SBS,런닝...",김소연 런닝맨 팬심 이상우 멤버들 리액션 반박해 180도 다들 거짓말 펜트하우스 1...,배우 김소연이 수줍은 팬심을 고백한다.\n오는 29일 오후 방송되는 SBS ‘런닝맨...,0
15,20201128,"‘트로트의 민족’ 김소연, 우승 후보→패자부활전 합격할까[M+TV인사이드]",문화>방송_연예,"트로트,민족,김소연,우승,후보,패자부활전,합격,TV인사이드,트로트,민족,유력,우승,...",트로트 박하명 긴장감 김소연 안성준 시청률 신명근 생존자 패자부활전 장명서 나미애 ...,‘트로트의 민족’ 유력 우승 후보 김소연이 패자부활전에 나선 가운데 그의 생존에 관...,0
21,20201128,"'아는 형님' 조병규, 즉석 액션부터 성대모사까지 매력 대방출!",문화>방송_연예,"형님,조병규,즉석,액션,성대모사,매력,대방출,배우,조병규,예능,블루칩,등극,방송,J...",조병규 김세정 눈물 성대모사 유준상 친밀도 만큼 촬영장 웃픈 블루칩 자유자재 전학생...,배우 조병규가 넘치는 끼로 예능 블루칩에 등극했다.\n \n28일 방송되는 JTBC...,0
67,20201128,‘온앤오프’ 이지아 “UFO 출몰 지역을 간 적도 있다”,문화>방송_연예,"온앤오프,이지아,UFO,출몰,지역,28일,방송,tvN,온앤오프,배우,이지아,반전,매...",이지아 온앤오프 덕후 ufo tvn 동작 하나하나 은하계 외계인설 싱어송라이터 놀라...,[헤럴드경제 = 서병기 선임기자]28일(토) 밤 방송되는 tvN ’온앤오프‘에서는 ...,0
...,...,...,...,...,...,...,...
19069,20201201,영화보며 시간 여행 자연 힐링 독서하며 언택트 송년,문화>방송_연예,"영화,여행,자연,힐링,독서,언택트,송년,연말,영화,리틀,포레스트,Little,개봉,...",구멍가게 일본 순천만 미국 곽재구 루나나 해남 목포 원스 혜원 보이저 시간여행 드루...,[연말에 보면 좋은 영화]\n▲리틀 포레스트(Little Forest /감독 임순례...,0
19304,20201201,"[종합] 아이랜드 통해 데뷔한 '엔하이픈' ""7명 전원이 센터급 비주얼"" 연말 신...",문화>방송_연예,"아이랜드,데뷔,엔하이픈,전원,센터,비주얼,연말,신인상,사진,빌리프랩,제공,빅히트엔터...",엔하이픈 아이랜드 빅히트 니키 희승 제이크 제이 타이틀곡 성훈 7명 신인상 팬들 방...,[사진= 빌리프랩 제공] \n \n \n 빅히트엔터테인먼트와 Mnet의 프로젝트 ...,0
19581,20201130,"[이웃사랑] 뇌병변에 유방암까지 온 아내, 홀로 고군분투 중인 남편",문화>방송_연예,"뇌병변,유방암,아내,고군분투,남편,결혼식,뇌병변,장애,남편,4년,병간호,유방암,생활...",주희 영석 결혼식 예주 유방암 중국집 대구은행 병간호 핸드폰 일자리 생활비 매일신문...,지난 27일 달서구 감삼동의 한 주택 2층. 쿵쾅쿵쾅 계단 위로 한 여자아이가 급히...,0
19598,20201130,전효성 다이어트 “3개월만에 허리 26인치→23.5인치”,문화>방송_연예,"3개월,전효,다이어트,허리,인치,23.5인치,가수,전효,다이어트,영상,공개,화제,전...",전효성 전효 3개월 다이어터 필라테스 배달음식 필요성 유튜브 장장 2개월 식이요법 ...,가수 전효성이 다이어트 영상을 공개해 화제다.\n\n\n\n전효성이 30일 오후 자...,0


In [16]:
NUM_LABELS = max(set(df['label'])) + 1
NUM_LABELS

58

In [17]:
text_to_label = dict(labels_csv)

In [18]:
label_to_text = {}
for label in text_to_label:
    label_to_text[int(text_to_label[label])] = label

In [19]:
label_to_text

{0: '문화>방송_연예',
 1: '국제>중국',
 2: '사회>사회일반',
 3: '정치>국회_정당',
 4: '사회>교육_시험',
 5: 'IT_과학>모바일',
 6: '정치>행정_자치',
 7: '사회>사건_사고',
 8: '문화>문화일반',
 9: '정치>청와대',
 10: '사회>여성',
 11: '국제>유럽_EU',
 12: '경제>자원',
 13: '국제>미국_북미',
 14: '사회>날씨',
 15: '사회>의료_건강',
 16: 'IT_과학>콘텐츠',
 17: '경제>증권_증시',
 18: '정치>선거',
 19: '국제>일본',
 20: '사회>미디어',
 21: '문화>음악',
 22: '국제>중남미',
 23: '국제>중동_아프리카',
 24: '경제>유통',
 25: '문화>출판',
 26: '경제>자동차',
 27: '사회>환경',
 28: '경제>금융_재테크',
 29: '사회>장애인',
 30: '문화>전시_공연',
 31: '경제>부동산',
 32: 'IT_과학>IT_과학일반',
 33: '문화>요리_여행',
 34: 'IT_과학>과학',
 35: '경제>서비스_쇼핑',
 36: '경제>외환',
 37: '미분류',
 38: '문화>미술_건축',
 39: '문화>생활',
 40: '경제>경제일반',
 41: '국제>아시아',
 42: '정치>정치일반',
 43: '경제>국제경제',
 44: '경제>취업_창업',
 45: 'IT_과학>보안',
 46: '정치>외교',
 47: '경제>무역',
 48: '문화>영화',
 49: '정치>북한',
 50: '경제>산업_기업',
 51: '문화>종교',
 52: '경제>반도체',
 53: '문화>학술_문화재',
 54: '사회>노동_복지',
 55: '국제>국제일반',
 56: 'IT_과학>인터넷_SNS',
 57: '국제>러시아'}

In [63]:
train, test = train, test = train_test_split(df, test_size=0.1)

print(train.shape)
print(test.shape)

(43631, 7)
(4848, 7)


In [64]:
train.head(10)

,date,title,topic,keyword,input,body,label
1441,20201127,[야고부] 문 정부의 두 걱정,정치>국회_정당,"정부,걱정,가지,걱정,나라,일자리,정치,인사,접촉,대구경북,기관장,여당,정치인,여당...",청와대 일자리 윤석열 대구경북 월성원전 기관장 정치인 법무부 문재인 추미애 수사지휘...,"""두 가지가 걱정입니다. 하나는 나라이고, 하나는 일자리입니다."" \n \n 정치권...",3
17745,20191127,"해양플랜트 수주, 내년에도 '꽁꽁'",경제>자원,"해양,플랜트,수주,내년,저유가,발주,조선3사,상선,사업,확대,나서,3사,조선,현대중...",해양플랜트 조선 현대중공업 유휴인력 삼성중공업 대우조선해양 양종서 연구원 한국 생산...,국내 조선 3사(현대중공업 대우조선해양 삼성중공업)의 해양플랜트 ‘수주 가뭄’이 올...,12
15240,20201120,"추미애 ""보수 언론이 윤석열과 갈등 부각 참을 수 없는 압통 느껴""",정치>청와대,"언론,추미애,보수,윤석열,갈등,부각,압통,장관,추미애,법무부,윤석열,검찰,총장,갈등...",법무부 윤석열 추미애 김홍영 압통 페이스북 부장검사 검찰개혁 여론몰이 꽃다발 가족들...,[머니투데이 김지영 기자] \n 추미애 법무부 장관이 최근 윤석열 검찰총장과의 갈등...,9
12633,20201204,"[브랜드 이야기-다이슨 V시리즈] 무선청소기의 신세계를 열다 ""가전계 애플""",경제>유통,"브랜드,이야기,다이슨,V시리즈,무선청소기,신세계,가전,애플,3M,스카치,테이프,사람...",다이슨 청소기 영국 소비자 ddm 무선청소기 컨슈머리포트 흡입력 한국 진공청소기 글...,3M을 모르는 이들은 있어도 '스카치 테이프'를 모르는 사람은 거의 없다. 소유주를...,24
18492,20201201,코로나19 확진자 증가속에도 경기북부 경찰 음주운전 사고 논란,사회>장애인,"증가속,코로나19,확진자,증가,논란,경기,북부,경찰,음주,운전,사고,모임,자제,정부...",음주운전 경기북부지방경찰청 경찰청 코로나19 운전자 확진자 관계자 연말연시 고양시 ...,모임 등을 자제해달라는 정부의 대국민 호소에도 국내에서 연일 400~500명대의 신...,29
12738,20201204,[이슈톡] '리얼돌'과 결혼한 카자흐스탄 남성,문화>방송_연예,"리얼돌,결혼,카자흐스탄,남성,가족,주변,만류,결혼,성공,커플들,감동,스토리,마련,이...",리얼돌 카자흐스탄 결혼식 이슈톡 보디빌더 여자친구 미지수 이목구비 코로나19 톨로츠...,가족과 주변의 만류에도 불구하고 결혼에 성공했다는 커플들을 보면 숨어 있는 감동 스...,0
9681,20201120,가상공간서 중장비 성능 테스트 한치의 오차 없는 크레인 만든다,IT_과학>인터넷_SNS,"가상공간,테스트,중장비,성능,오차,크레인,타워크레인,중장비,제작,고강도,기본,오차,...",중장비 hil 프라운호퍼 연구소 지브 응용기술 스마트공장 시제품 독일 5g 유럽 네...,"타워크레인 등 중장비는 제작 과정이 복잡하다. 고강도는 기본이고, 한치의 오차 없이...",56
16352,20201203,“힘 합치겠다”던 여야 ‘중대재해법’ 내용 따져보니,정치>국회_정당,"여야,중대재해법,앵커,정기국회,엿새,법안,중대,재해법,제정,정의당,3일,국회,농성,...",정의당 정기국회 민주당 사업주 공무원 중대재해법 김종인 김용균 중대재해 박주민 법안...,[앵커]\n\n정기국회가 이제 엿새 남았습니다.\n\n1호 법안으로 중대재해법 제정...,3
16730,20201120,이동걸 “통합 무산땐 아시아나 국유화 더 큰 문제 야기될 것”,경제>산업_기업,"국유화,이동걸,통합,무산,아시아나,국유,야기,확보,항공업,경쟁력,특혜논란,조원태,회...",한진칼 아시아나항공 위원회 대한항공 이동걸 산은 의원들 kcgi 구조조정 조원태 국...,정치권 등에서 대한항공과 아시아나항공 통합 결정을 둘러싼 특혜 시비가 일고 있는 가...,50
3560,20201204,봉쇄 강도 높인 北 “상주 외교관 국제기구 직원 상당수 철수”,정치>북한,"봉쇄,강도,외교관,상주,국제기구,직원,철수,직원,대변,이크발,ICRC,평양,사무소,...",평양 대사관 코로나 대변인 외국인 중국 국제기구 사무소 미국 icrc 상주 외국인들...,북한에 파견돼 대북 지원업무를 하던 국제적십자위원회(ICRC) 관계자들이 2일 현지...,49


<br>
<br>

# **Preprocessing - TRAINING SET**

In [65]:
# 문장 추출
sentences = train['input'].values
sentences[:10]

array(['청와대 일자리 윤석열 대구경북 월성원전 기관장 정치인 법무부 문재인 추미애 수사지휘권 쓴소리 失政 풀칠 인사권 감찰권 서울 국회의원 부산 부산시장 보궐선거 따름 박근혜 조응천 울산 감사원',
       '해양플랜트 조선 현대중공업 유휴인력 삼성중공업 대우조선해양 양종서 연구원 한국 생산직 아랍에미리트 국제유가 부유식 한국투자증권 재배치 상선건조 000여명',
       '법무부 윤석열 추미애 김홍영 압통 페이스북 부장검사 검찰개혁 여론몰이 꽃다발 가족들 이해타산 보수언론 마음들 대검찰청 대한민국 장관 검사 마음 소명 총장 심정 가족 사람 선택 자리 강조 감찰 국민 완수 보수',
       '다이슨 청소기 영국 소비자 ddm 무선청소기 컨슈머리포트 흡입력 한국 진공청소기 글라이드 옴니 리버풀 고장률 그동안 데이비드 v11',
       '음주운전 경기북부지방경찰청 경찰청 코로나19 운전자 확진자 관계자 연말연시 고양시 코로나바이러스 대국민 기강해이 감염증 내부직원들 일산서구 혈중알코올농도 경찰관 서울 운전대 증가속',
       '리얼돌 카자흐스탄 결혼식 이슈톡 보디빌더 여자친구 미지수 이목구비 코로나19 톨로츠코 18세 커플들 마고 결혼 부부 인형 커플 조항 가족 아내 유리 법적 성공 여행 친구 추억 모습 여자 전망 사람 스토리 투데이 마련 확산 진짜 외모 현지',
       '중장비 hil 프라운호퍼 연구소 지브 응용기술 스마트공장 시제품 독일 5g 유럽 네트워크 방정식 가상공간 프라운호퍼itwm itwm 과학기술정보통신부 하드웨어',
       '정의당 정기국회 민주당 사업주 공무원 중대재해법 김종인 김용균 중대재해 박주민 법안들 공청회 재해법 강은미 강정희 영상편집 김민준 노동수 위원장 기잡 촬영기자 더불어민주당 허가권 손해배상 김빛 사망사고 사실상 10억 원내대표 국민의힘',
       '한진칼 아시아나항공 위원회 대한항공 이동걸 산은 의원들 kcgi 구조조정 조원태 국유화 항공사 조선사 경쟁력 강성부 경영권 아시아나 신주발

In [66]:
# 라벨 추출
labels = list(map(str, train['label'].values))
labels[:10]

['3', '12', '9', '24', '29', '0', '56', '3', '50', '49']

In [67]:
train_dataset = pd.DataFrame({'sentence': sentences, 'label': labels}, columns=['sentence', 'label'])

<br>
<br>

# **Preprocessing - TEST SET**

In [68]:
# 문장 추출
sentences = test['input'].values
sentences[:10]

array(['순두부 표고버섯 새우젓 네츄르먼트 약간씩 홍고추 풋고추 2개 4대 2컵 모시조개 후춧가루 1봉 2인 시간date 레시피 순백 요리 사진 마늘 대파 바지락 재료 찌개 소금 제공',
       '홍콩 관광객 코로나19 중국 눈물 scmp 본토인 쇼핑몰 쇼핑몰들 뉴질랜드 관계자 태국 호주 소매점 국가들 소비자 지원금 2주 일본',
       '온라인 차은우 여신강림 tvn yalbr 포토타임 코로나19 발표회 취재진 사진제공 제작발표회 배우 가수 타임 포토 행사 영향 수목 제작 사진 드라마 제공 출입 진행 화보',
       '부동산 전주 5억 투기세력 감나무골 11억 4억 입주가액 실수요자 노동식 매매가액 공인중개사 분양가',
       '중국 중고차 현대글로비스 자동차 창지우 합자회사 해운 서울 역삼동 김성훈 동아시아 화주사 중국발 김정훈',
       '김윤석 평가관 백혈병 코로나19 헌혈증 골수 전문평가관 유전자형 유전자 부대원 공감대 군무원 명예대장 헌혈 부족 육군 이식 유공 전문 수상 대장 건강 확산 일치 친척 보름 명예 소식 관리',
       '코로나19 경북대병원 대구 음압기 제왕절개 감염병 코로나바이러스 a씨 수술실 의료진 학정동 칠곡경북대병원 경북 관리청 제왕절개수술 방호복',
       '수험생 안내문 예산안 정부안 격리중 코로나 규몹 내년도 시험장 수험표 본회의 예비 수능 처리 수험 소집 법정 일정 여야 전달 선진 합의 화법 확진 예산 가족 하루 전국 국회 사항 유의 시험 내년 직계 격리',
       '서울성모병원 허양 수험생 감독관 합병증 시험장 1주일 교육청 코로나바이러스 김동욱 가톨릭대학교 무기력감 고사장 적혈구 듣기평가 백혈구 조혈모세포 의과대학 코로나19 4명 감염증 감독관들 혈액세포 골수 최소화 서울시교육청 응급조치',
       '추모식 원내대표 김영삼도서관 김종인 박병석 더불어민주당 김태년 서울 상도동 정세균 위원장 서울현충원 국무총리 주호영 참석자들 김영삼 개관식 참석자 참석 대통령 대표 원내 

In [69]:
# 라벨 추출
labels = list(map(str, test['label'].values))
labels[:10]

['39', '1', '0', '31', '1', '2', '15', '6', '15', '9']

In [70]:
test_dataset = pd.DataFrame({'sentence': sentences, 'label': labels}, columns=['sentence', 'label'])

In [71]:
dataset_train = train_dataset.values
dataset_test = test_dataset.values

In [72]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [30]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = []
        self.labels = []
        
        for i in dataset:
            try:
                self.sentences.append(transform([i[sent_idx]]))
                self.labels.append(np.int32(i[label_idx]))
            except Exception as e:
                print(e)
                print(len(self.sentences), len(self.labels))

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [39]:
## Setting parameters
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  2e-5

In [73]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

Argument 'text' has incorrect type (expected str, got float)
5036 5036
Argument 'text' has incorrect type (expected str, got float)
19364 19364
Argument 'text' has incorrect type (expected str, got float)
26865 26865
Argument 'text' has incorrect type (expected str, got float)
29137 29137
Argument 'text' has incorrect type (expected str, got float)
41011 41011
Argument 'text' has incorrect type (expected str, got float)
2986 2986
Argument 'text' has incorrect type (expected str, got float)
3827 3827


In [74]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [50]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=NUM_LABELS,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [51]:
model = BERTClassifier(bertmodel,  dr_rate=0.25).to(device)

In [52]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [53]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [54]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [55]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [56]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [75]:
from time import gmtime, strftime
START_TIME = strftime("%Y_%m_%d_%H_%M_%S", gmtime())

In [76]:
SAVE_PATH = BASE_PATH+"/bert_news_model_{}.pth".format(START_TIME)

In [77]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    torch.save(model.state_dict(), SAVE_PATH)
    print("Model saved at {}".format(SAVE_PATH))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 0.8635578155517578 train acc 0.765625
epoch 1 batch id 201 loss 0.731905996799469 train acc 0.7902674129353234
epoch 1 batch id 401 loss 0.8446386456489563 train acc 0.7874844139650873
epoch 1 batch id 601 loss 0.8564671874046326 train acc 0.7865796589018302

epoch 1 train acc 0.7857481060606061
Model saved at /content/drive/MyDrive/hyun_data/bert_news_model_2020_12_13_04_55_15.pth



epoch 1 test acc 0.811570366132723


epoch 2 batch id 1 loss 0.8304495811462402 train acc 0.765625
epoch 2 batch id 201 loss 0.8136337399482727 train acc 0.789179104477612
epoch 2 batch id 401 loss 0.8513218760490417 train acc 0.7902119700748129
epoch 2 batch id 601 loss 0.8542167544364929 train acc 0.788529534109817

epoch 2 train acc 0.7886686566122051
Model saved at /content/drive/MyDrive/hyun_data/bert_news_model_2020_12_13_04_55_15.pth



epoch 2 test acc 0.8058137871853547


epoch 3 batch id 1 loss 0.9465353488922119 train acc 0.734375
epoch 3 batch id 201 loss 0.7795008420944214 train acc 0.7913557213930348
epoch 3 batch id 401 loss 0.8350229263305664 train acc 0.793251246882793
epoch 3 batch id 601 loss 0.6745178699493408 train acc 0.7928192595673876

epoch 3 train acc 0.7926223641949449
Model saved at /content/drive/MyDrive/hyun_data/bert_news_model_2020_12_13_04_55_15.pth


...


epoch 8 test acc 0.7595198083524027


epoch 9 batch id 1 loss 0.39328306913375854 train acc 0.90625
epoch 9 batch id 201 loss 0.457494854927063 train acc 0.8894589552238806
epoch 9 batch id 401 loss 0.3453103005886078 train acc 0.8917160224438903
epoch 9 batch id 601 loss 0.3180490732192993 train acc 0.8948367304492513

epoch 9 train acc 0.8956195014662757
Model saved at /content/drive/MyDrive/hyun_data/bert_news_model_2020_12_13_04_55_15.pth



epoch 9 test acc 0.7687714530892449


epoch 10 batch id 1 loss 0.48757418990135193 train acc 0.875
epoch 10 batch id 201 loss 0.3737635016441345 train acc 0.9111473880597015
epoch 10 batch id 401 loss 0.44737380743026733 train acc 0.9122895885286783
epoch 10 batch id 601 loss 0.1455303132534027 train acc 0.9145174708818635

epoch 10 train acc 0.9149778313084764
Model saved at /content/drive/MyDrive/hyun_data/bert_news_model_2020_12_13_04_55_15.pth



epoch 10 test acc 0.7638372425629291


In [61]:
import gc
gc.collect()

84